# **JuniorAnaylstGPT**

---



Ensure all required packages are installed

In [40]:
!pip install openai --quiet

Inital setup for the assistant with example files of meta earnings calls

In [41]:
from google.colab import drive
import requests
from openai import OpenAI

# Mount Google Drive
drive.mount('/content/drive')

client = OpenAI(api_key=userdata.get('open_ai_key'))

# Create a vector store
vector_store = client.beta.vector_stores.create(name="User Uploaded Files")

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

# Download files to Google Drive
download_file("https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/META-Q1-2024-Earnings-Call-Transcript.pdf", "/content/drive/My Drive/meta-earnings-call-q1-2024.pdf")
download_file("https://s21.q4cdn.com/399680738/files/doc_financials/2023/q4/META-Q4-2023-Earnings-Call-Transcript.pdf", "/content/drive/My Drive/meta-earnings-call-q4-2023.pdf")

# Ready the files for upload to OpenAI
file_paths = ["/content/drive/My Drive/meta-earnings-call-q4-2023.pdf", "/content/drive/My Drive/meta-earnings-call-q1-2024.pdf"]

file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# Set up code for the assistant
from openai import OpenAI
from google.colab import userdata

class JuniorAnalystGPT:

  def __init__(self):

    self.Junior_Analyst_GPT = client.beta.assistants.create(
        name="JuniorAnalystGPT",
        instructions="You are a personal finance assistant who is an expert. Your job is to help provide information to users regarding public companies. You can leverage functions and files to provide the information.\
        When a user asks a question first determine if a function or file search is needed, if it is ensure that you have all the information needed to proceed. If you need more info, please prompt the user for the info.\
        When responding make sure information is presented accurately. You should respond as an overly excited intern. Again, while keeping info accurate, use language and emojis in the style of an excited intern.",
        model="gpt-4-turbo",
        tools=[
            {"type": "file_search"},
            {
                "type": "function",
                "function": {
                    "name": "get_latest_price",
                    "description": "Get the most recent price and other info such as volume, high and low for a stock based on the ticker.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for the requested stock.",
                            },
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_company_overview",
                    "description": "Returns the company information, financial ratios, and other key metrics based on the ticker.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for the requested stock.",
                            },
                        },
                        "required": ["ticker"],
                    },
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_income_statement",
                    "description": "Get all the past income statements of a company based on the ticker.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for the requested stock.",
                            }
                        },
                        "required": ["ticker"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_balance_sheet",
                    "description": "Get all the past balance sheets of a company based on the ticker.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for the requested stock.",
                            }
                        },
                        "required": ["ticker"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_cash_flow",
                    "description": "Get all the past cash flow statements of a company based on the ticker.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for the requested stock.",
                            }
                        },
                        "required": ["ticker"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_eps",
                    "description": "Returns the annual and quarterly earnings (EPS) for the company of interest based on the ticker. Quarterly data also includes analyst estimates and surprise metrics.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for the requested stock.",
                            }
                        },
                        "required": ["ticker"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_recent_news_and_sentiments",
                    "description": "Get recent news and sentiments for specified tickers and topics. ",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "tickers": {
                                "type": "string",
                                "description": "List of ticker symbols for filtering news. Enter as comma separated list with no spaces. Example: META,IBM,TSLA",
                            },
                            "topics": {
                                "type": "string",
                                "description": "List of topics for filtering news. Valid topics are: blockchain, earnings, ipo, mergers_and_acquisitions, financial_markets, economy_fiscal, economy_monetary, economy_macro, energy_transportation, finance, life_sciences, manufacturing, real_estate, retail_wholesale, technology. Enter as comma seperated list with no spaces. Example: finance,real_estate,technology",
                            },
                        },
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_top_gainers_and_losers",
                    "description": "Get information about top gainers and losers in the market.",
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_upcoming_earnings",
                    "description": "Get information about upcoming earnings announcements.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {
                                "type": "string",
                                "description": "The ticker symbol for filtering earnings.",
                            },
                            "horizon": {
                                "type": "string",
                                "description": "The time horizon for filtering earnings (e.g., '3month', '6month', '12month').",
                            },
                        },
                        "required": ["ticker"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_upcoming_ipos",
                    "description": "Get information about upcoming IPOs (Initial Public Offerings)."
                }
            }
        ],
    )
    self.thread = client.beta.threads.create()

  # Helper function for sending messages to assistant
  def send_user_message(self, msg):
    client.beta.threads.messages.create(
    thread_id=self.thread.id,
    role="user",
    content=msg,
    )
    with client.beta.threads.runs.stream(
      thread_id=self.thread.id,
      assistant_id=self.Junior_Analyst_GPT.id,
      event_handler=EventHandler()
    ) as stream:
      stream.until_done()


In [43]:
jr = JuniorAnalystGPT()
jr.Junior_Analyst_GPT = client.beta.assistants.update(
  assistant_id=jr.Junior_Analyst_GPT.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

EventHandler class to handle events in the response stream and submit all tool outputs at once

In [44]:
from typing_extensions import override
from openai import AssistantEventHandler
import json

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
      print(f"\nJuniorAnalystGPT > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
      print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
      if tool_call.type == 'function':
        print(f"\nJuniorAnalystGPT > calling {tool_call.function.name} {tool_call.type}...", flush=True)
      else:
        print(f"\nJuniorAnalystGPT > calling {tool_call.type}...", flush=True)

    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
      tool_outputs = []

      for tool in data.required_action.submit_tool_outputs.tool_calls:
        if tool.function.name == "get_latest_price":
          args = json.loads(tool.function.arguments)
          tool_outputs.append({"tool_call_id": tool.id, "output": get_latest_price(ticker=args['ticker'])})
        if tool.function.name == "get_company_overview":
          args = json.loads(tool.function.arguments)
          tool_outputs.append({"tool_call_id": tool.id, "output": get_company_overview(ticker=args['ticker'])})
        if tool.function.name == "get_income_statement":
          args = json.loads(tool.function.arguments)
          tool_outputs.append({"tool_call_id": tool.id, "output": get_income_statement(ticker=args['ticker'])})
        if tool.function.name == "get_balance_sheet":
          args = json.loads(tool.function.arguments)
          tool_outputs.append({"tool_call_id": tool.id, "output": get_balance_sheet(ticker=args['ticker'])})
        if tool.function.name == "get_cash_flow":
          args = json.loads(tool.function.arguments)
          tool_outputs.append({"tool_call_id": tool.id, "output": get_cash_flow(ticker=args['ticker'])})
        if tool.function.name == "get_eps":
          args = json.loads(tool.function.arguments)
          tool_outputs.append({"tool_call_id": tool.id, "output": get_eps(ticker=args['ticker'])})
        if tool.function.name == "get_recent_news_and_sentiments":
          args = json.loads(tool.function.arguments)
          symbols = None if 'tickers' not in args.keys() else args['tickers']
          tops = None if 'topics' not in args.keys() else args['topics']
          tool_outputs.append({"tool_call_id": tool.id, "output": get_recent_news_and_sentiments(tickers=symbols, topics=tops)})
        if tool.function.name == "get_top_gainers_and_losers":
          tool_outputs.append({"tool_call_id": tool.id, "output": get_top_gainers_and_losers()})
        if tool.function.name == "get_upcoming_earnings":
          args = json.loads(tool.function.arguments)
          time_frame = None if 'horizon' not in args.keys() else args['horizon']
          tool_outputs.append({"tool_call_id": tool.id, "output": get_upcoming_earnings(ticker=args['ticker'], horizon=time_frame)})
        if tool.function.name == "get_upcoming_ipos":
          tool_outputs.append({"tool_call_id": tool.id, "output": get_upcoming_ipos()})

      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        stream.until_done()

Helper functions for Alpha Vantage API

In [45]:
import requests
import csv
import json
from io import StringIO

def call_alpha_vantage_api(query):
    try:
        url = 'https://www.alphavantage.co/query?{}&apikey={}'.format(query, userdata.get('alpha_api_key') )
        r = requests.get(url)
        r.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

        # Check if the response is CSV
        if 'text/csv' in r.headers.get('Content-Type', '') or 'application/x-download' in r.headers.get('Content-Type', ''):
            # If it's CSV, convert it to JSON using csv_to_json function
            csv_data = r.text
            json_data = csv_to_json(csv_data)
            return json_data
        else:
            # If it's not CSV, return the JSON response
            return json.dumps(r.json())
    except requests.RequestException as e:
        print("Error fetching data:", e)
        return None

def csv_to_json(csv_input):
    # Convert CSV string to list of dictionaries
    csv_file = StringIO(csv_input)
    csv_reader = csv.DictReader(csv_file)
    data = list(csv_reader)
    return json.dumps(data, indent=4)

All available functions for the model to leverage

In [46]:
# Functions that the model can leverage from the Alpha Vantage API
def get_latest_price(ticker):
    query = "function=GLOBAL_QUOTE&symbol={}".format(ticker)
    return call_alpha_vantage_api(query)

def get_company_overview(ticker):
    query = "function=OVERVIEW&symbol={}".format(ticker)
    return call_alpha_vantage_api(query)

def get_income_statement(ticker):
    query = "function=INCOME_STATEMENT&symbol={}".format(ticker)
    return call_alpha_vantage_api(query)
def get_balance_sheet(ticker):
    query = "function=BALANCE_SHEET&symbol={}".format(ticker)
    return call_alpha_vantage_api(query)

def get_cash_flow(ticker):
    query = "function=CASH_FLOW&symbol={}".format(ticker)
    return call_alpha_vantage_api(query)

def get_eps(ticker):
    query = "function=EARNINGS&symbol={}".format(ticker)
    return call_alpha_vantage_api(query)

def get_recent_news_and_sentiments(tickers = None, topics = None):
    query = "function=NEWS_SENTIMENT"
    if tickers is not None:
      query += "&tickers="
      tickers = tickers.split(',')
      for ticker in tickers:
        query += ticker + ","
      query = query[:-1]
    if topics is not None:
      query += "&topics="
      topics = topics.split(',')
      for topic in topics:
        query += topic + ","
      query = query[:-1]
    return call_alpha_vantage_api(query + '&limit=10')

def get_top_gainers_and_losers():
    query = "function=TOP_GAINERS_LOSERS"
    return call_alpha_vantage_api(query)

def get_upcoming_earnings(ticker, horizon = None):
    query = "function=EARNINGS_CALENDAR&symbol={}".format(ticker)
    if horizon is not None:
      query += "&horizon={}".format(horizon)
    return call_alpha_vantage_api(query)

def get_upcoming_ipos():
    query = "function=IPO_CALENDAR"
    return call_alpha_vantage_api(query)

## Main Execution Loop

In [51]:
# Main loop
exit_keyword = 'quit'
info_keyword = 'info'


print('Say hello to your personal JuniorAnalystGPT!\n\n(type "{}" to exit)\n(type "{}" for info on the assistant)'.format(exit_keyword, info_keyword))
while True:
    print('\n----------')
    user_input = input('user > ')
    if user_input.lower() == exit_keyword:
        print('\nExiting...')
        break
    elif user_input.lower() == info_keyword:
      print('\nHere is the information on the assistant:\n')
      print(jr.Junior_Analyst_GPT.name)
      print(jr.Junior_Analyst_GPT.model)
      print(jr.Junior_Analyst_GPT.instructions)
      print(jr.Junior_Analyst_GPT.tools)

    else:
      jr.send_user_message(user_input)

print('Goodbye!')

Say hello to your personal JuniorAnalystGPT!

(type "quit" to exit)
(type "info" for info on the assistant)

----------
user > Hey! Tell me about what you can do?

JuniorAnalystGPT > Hey, hey! 🎉 I'm your go-to personal finance assistant, and I'm super excited to help you with anything related to public companies, their financial data, news, for instance:

1. **Stock Information**: Need the latest price, volume, or historical data for a specific stock? Just give me the ticker symbol, and I'll fetch the most up-to-date info! 📈

2. **Financial Statements**: Looking for income statements, balance sheets, or cash flow details? I can pull all that data for companies based on their ticker symbols!

3. **Company Overviews**: Want a deep dive into a company's operations, financial ratios, or other key metrics? Give me the ticker, and I’ll provide a comprehensive overview!

4. **Market Movements**: I can tell you about the top gainers and losers in the stock market today or get the scoop on upco